In [ ]:
import os
from datetime import datetime, timedelta
from geopandas import GeoDataFrame
import geopandas as gpd
import gpd_lite_toolbox as glt
import pointpats.quadrat_statistics as qs
from pointpats import PointPattern
import pysal as ps
import shapely.speedups
import pandas as pd
from sklearn.cluster import DBSCAN, OPTICS
import shapely.geometry
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import contextily as ctx
import zipfile


plt.rcParams["font.family"] = "Times New Roman"
shapely.speedups.enable()
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = [12.0, 12.0]

In [ ]:
def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

In [ ]:
def pip_count(points, polygon):
    try : 
        pips = points.within(polygon).value_counts()[True]
    except:
        pips = 0
    
    return pips

# Data Prep

In [ ]:
zf = '/Users/GeorgePyne/Documents/CASA/Digital Visualisation/Group Project/Foursquare_Data.zip'
csvs = [i for i in zipfile.ZipFile(zf).namelist() if str(i).endswith('csv')]
with zipfile.ZipFile(zf) as zipf:
    with zipf.open(csvs[0]) as myZip:
        df = pd.read_csv(myZip)

In [ ]:
import zipfile
zf = '/Users/GeorgePyne/Downloads/dataset_tsmc2014.zip'
csvs = [i for i in zipfile.ZipFile(zf).namelist() if str(i).endswith('csv')]


In [ ]:
uk = df.loc[df['country_code']=='GB']
df = None

In [ ]:
uk['geometry'] = [shapely.geometry.Point(lat,lon) for lat,lon in zip(uk.lat,uk.lon)]
uk = GeoDataFrame(uk)
# pd.to_datetime(uk.datetime)
uk = GeoDataFrame(uk)
uk.crs = {'init': 'epsg:4326'}
uk = uk.to_crs(epsg=27700)

bb = gpd.read_file('/Users/GeorgePyne/Documents/Geography/2nd Year/Spatial Analysis/5SSG2060_CW1_1531870/data/LDN-LSOAs/LDN-LSOAs.shp')
bb = bb['geometry']
bb.crs = {'init': 'epsg:27700'}

xmin, ymin, xmax, ymax = bb.total_bounds
lon = uk.cx[xmin:xmax, ymin:ymax]
uk = None
lon['datetime'] = pd.to_datetime(lon['datetime'])
lon = lon.reset_index(drop=True)
lon['lon'],lon['lat'] = lon['geometry'].apply(lambda x: x.x), lon['geometry'].apply(lambda x: x.y)

In [ ]:
lon.groupby('venue_type').count().sort_values(by='user_id', ascending=False).head(20)

In [ ]:
base = lon.datetime[0]
lon['time'] = lon.datetime - base
lon = lon.sort_values(by='time')
lon['time'] = lon['time'].apply(lambda x: (x.total_seconds() / 60)).astype(int)

In [ ]:
def decomp_preperation(df):
    """Prepares the data for ST decomposition in XYT format"""
    filename = input('Filename? ')
    directory = "/Users/GeorgePyne/Documents/CASA/Dissertation/IPYNB/decompSpaceTime-master/files/"+filename
    
    df[['lon','lat','time']].to_csv(directory,header=False,index=False)
    xmin, xmax = df.lon.min(), df.lon.max() 
    ymin, ymax = df.lat.min(), df.lat.max()
    zmin, zmax = df.time.min(), df.time.max()
        
    src = open(directory,"r")
    fline = '{0},{1},{2},{3},{4},{5}\n'.format(xmin,xmax,ymin,ymax,zmin,zmax)
    oline = src.readlines()
    oline.insert(0,fline)
    src.close()
    
    src=open(directory,"w")
    src.writelines(oline)
    src.close()

In [ ]:
decomp_preperation(lon)

# Spatiotemporal Decomposition

In [ ]:
pList

In [ ]:
#import modules
from datetime import datetime
import sys, os
import decomposition as decomp, settings as sett
import geopandas as gpd
import pandas as pd
import shapely
from shapely import geometry
import shutil

#set recursion limit
sys.setrecursionlimit(8000)

#initialize global variables
sett.init()
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#read parameters
pFile = open('files/parameterFile.txt', "r")
pFile.readline()
pList = pFile.readline().split("\t")

sett.p1 = float(pList[0])	# p1 = spatial bandwidth
sett.p2 = float(pList[1])	# p2 = temporal bandwidth
sett.p3 = float(pList[2])	# p3 = spatial resolution
sett.p4 = float(pList[3])	# p4 = temporal resolution
sett.p5 = float(pList[4])	# p5 = number of points threshold (T1)
sett.p6 = float(pList[5])	# p6 = buffer ratio threshold (T2)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#create output directory
sett.dir1 = 'pointFiles'
sett.dir2 = 'timeFiles'
sett.dir3 = 'shpFiles'

if os.path.exists(sett.dir1):
    shutil.rmtree(sett.dir1)
os.makedirs(sett.dir1)

if os.path.exists(sett.dir2):
    shutil.rmtree(sett.dir2)
os.makedirs(sett.dir2)

if os.path.exists(sett.dir3):
    shutil.rmtree(sett.dir3)
os.makedirs(sett.dir3)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#read input point file
pFile = open('files/decomp_test_prep.txt', "r")
inX, inY, inZ = [], [], []
r = pFile.readline().split(",")
xmin, xmax, ymin, ymax, zmin, zmax = float(r[0]), float(r[1]), float(r[2]), float(r[3]), float(r[4]), float(r[5].strip())

for record in pFile:   
    inX.append(float(record.split(",")[0]))
    inY.append(float(record.split(",")[1]))
    inZ.append(float(record.split(",")[2]))
    
pFile.close()
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#start decomposition
startTime = datetime.now()
decomp.decompose(inX, inY, inZ, xmin, xmax, ymin, ymax, zmin, zmax)
endTime = datetime.now()

#record decomposition time
runTime = endTime - startTime 
print("Decomposition into {0} files in {1}.".format(len(os.listdir('pointFiles')), runTime))
tFile=open('timeFiles/decomp_time.txt', "w")
tFile.write(str(runTime))
tFile.close()
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Methodology

In [ ]:
cluster_spots, cluster_time = [],[]
static_frames, moving_frames = [], []
static_frames_t, moving_frames_t = [], []
lm_list = []
lm_moves_list = []
# txts = []
times = []
moving_frame_t = []

directory = '/Users/GeorgePyne/Documents/CASA/Dissertation/IPYNB/decompSpaceTime-master/pointFiles'

for filename in os.listdir(directory)[0:3]: # Iterate over each decomposition domain
    txt = pd.read_csv(directory+'/'+filename, skiprows=1, names=['lon','lat','time']) # read decomp
#     txts.append(txt)
    points = [[lat,lon] for lat,lon in zip(txt.lat, txt.lon)] # Save point patter to test global CSR
    pp = PointPattern(points) # Create point pattern object
    domain_quadrat = qs.QStatistic(pp, shape= "rectangle", nx = 6, ny = 7) # Run quadrat analysis on point pattern object
#     domain_quadrat.plot()
    pv = float(str(domain_quadrat.chi2_pvalue)[0:4])
    if pv < 1.0: # If not CSR and if statistically significant
        start_time = datetime.now() # Time subdomain process
        
        frames = []
        counts = []

        # Create geo_df of points to rasterize
        points_gdf = GeoDataFrame([shapely.geometry.Point(point) for point in points]).rename(columns={0:'geometry'})
        xmin,ymin,xmax,ymax = points_gdf.total_bounds # Get point bounds
        height = (xmax-xmin)/ 6 # Set grid resolution
        grid = glt.make_grid(points_gdf,height, False) # Create unclipped grid   
        
        
        for t in txt.time.unique(): # Iterate over each frame
            frame = txt.loc[txt.time == t] # create frame of each time step
            points = [[lat,lon] for lat,lon in zip(frame.lat, frame.lon)] # make list of points
            frame['geometry'] = [shapely.geometry.Point(point) for point in points] # make point df
            frame = GeoDataFrame(frame) # change to gdf
            frames.append(frame)
            grid[str(int(t))+'_count'] = grid.geometry.apply(lambda x: pip_count(frame,x)) # Aggregate points to grid

        W = ps.weights.Queen.from_dataframe(grid) # Calculate spatial Queen contiguity weights object from grid gdf 
        tci = np.array(grid[grid.columns[1:]]) # Save transitions as matrix
        lm = ps.LISA_Markov(tci,W) # Calculate LISA markov transitions from W and matrix
        lm_list.append(lm)
        
        # creat a LISA transition df of observed against expected
        lm_moves = pd.DataFrame({"Transitions":lm.transitions.flatten(), "Expected":lm.expected_t.flatten()}) 
        lm_moves['Transitions'] = lm_moves['Transitions'].astype(int)
        lm_moves.index = ["HH-HH","HH-LH","HH-LL","HH-HL", # Make index 16 possible move types
                  "LH-HH","LH-LH","LH-LL","LH-HL",
                  "LL-HH","LL-LH","LL-LL","LL-HL",
                  "HL-HH","HL-LH","HL-LL","HL-HL"]
        lm_moves['Residuals'] = lm_moves.Transitions - lm_moves.Expected # Calculate difference of observed - expected
        lm_moves_list.append(lm_moves)
    
        if lm.chi_2[1] < 0.05:
            for frame in frames:                
                if len(frame)>2:
                    points = [[lat,lon] for lat,lon in zip(frame.lat, frame.lon)]
                    pp_t = PointPattern(points)
                    q_h_t = qs.QStatistic(pp,shape= "rectangle",nx = 6, ny = 7)
                    pv_t = float(str(q_h_t.chi2_pvalue)[0:4])
                    if pv_t < 1.0:
                        eps = pp_t.mean_nnd            
                        min_samples = int(len(frame) / 6)
                    if min_samples < 3:
                        min_samples = 3
                    labels = OPTICS(eps=eps, min_samples=min_samples).fit(points).labels_
                    frame['labels'] = labels
                    try:
                        for i in frame.labels.unique():
                            if i > -1:
                                if len(frame.loc[frame['labels']==i]) > 2:
                                    moving_frames.append\
                                    (shapely.geometry.MultiPoint\
                                     ([shapely.geometry.Point(lat,lon)\
                                       for lat,lon in zip(frame.loc[frame['labels']==i].lon,\
                                                          frame.loc[frame['labels']==i].lat)]).convex_hull)
                                    moving_frame_t.append(frame.time.unique()[0])
                                    end_time = datetime.now() # Save endtime
                                    run_time = end_time - start_time # Find runtime to adjudge parallelization
                                    times.append(run_time)
                    except:
                        print(frame.head(4))
                        
        else:
            for frame in frames:  
                if len(frame)>4:
                    points = [[lat,lon] for lat,lon in zip(frame.lat, frame.lon)]
                    pp_t = PointPattern(points)
                    q_h_t = qs.QStatistic(pp,shape= "rectangle",nx = 6, ny = 7)
                    pv_t = float(str(q_h_t.chi2_pvalue)[0:4])
                    if pv_t < 1.0:
                        eps = pp_t.mean_nnd            
                        min_samples = int(len(frame) / 6)
                    if min_samples < 3:
                        min_samples = 3
                    labels = OPTICS(eps=eps, min_samples=min_samples).fit(points).labels_
                    frame['labels'] = labels
                    try:                        
                        for i in frame.labels.unique():                            
                            if i > -1:
                                if len(frame.loc[frame['labels']==i]) > 2:                                   
                                    static_frames.append(shapely.geometry.MultiPoint([shapely.geometry.Point(lat,lon) for lat,lon in zip(frame.loc[frame['labels']==i].lon, frame.loc[frame['labels']==i].lat)]).convex_hull)
                                    static_frames_t.append(frame.time.unique()[0])
                                    end_time = datetime.now() # Save endtime
                                    run_time = end_time - start_time # Find runtime to adjudge parallelization
                                    times.append(run_time)
                    except:
                        print(frame.head(4))
                
        end_time = datetime.now()
        run_time = end_time - start_time
        times.append(run_time)
    else:
        print("CSR expected in {}.".format(filename))

In [ ]:
txt = txt.sort_values(by='time')
txt['framerate'] = txt.time.shift(1)
frame_rate = txt[['time','framerate']].diff().framerate.mean()
txt['framerate'] = (txt.framerate / frame_rate).fillna(method='backfill').astype(int)
txt.time = txt.framerate * frame_rate

In [ ]:
def graph_run_times(times):
    

In [ ]:
import geopandas as gpd
bb = gpd.read_file('/Users/GeorgePyne/Documents/Geography/2nd Year/Spatial Analysis/5SSG2060_CW1_1531870/data/LDN-LSOAs/LDN-LSOAs.shp')
bb = bb['geometry']
bb.crs = {'init': 'epsg:27700'}
bb = bb.to_crs(epsg=3857)
xmin, ymin, xmax, ymax = bb.total_bounds

In [ ]:
moving_clusters['minutes'] = moving_clusters.time * 30
moving_clusters['datetime'] = moving_clusters.minutes.apply(lambda x: ts + timedelta(minutes=x))
moving_clusters = moving_clusters.sort_values(by='time', ascending=True)
moving_clusters.crs = {'init': 'epsg:27700'}
moving_clusters = moving_clusters.to_crs(epsg=3857)
for time in moving_clusters.time.unique():
    ax = moving_clusters.loc[moving_clusters['time']==time].plot(cmap='OrRd')
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.set_axis_off()
    add_basemap(ax, zoom=13, url=ctx.sources.ST_TONER_LITE)
#     plt.show()

In [ ]:
moving_clusters = GeoDataFrame(moving_frames).rename(columns={0:'geometry'})
moving_clusters['time'] = moving_frame_t
moving_clusters.time.unique()

In [ ]:
LISA_markov_diagnostics(lm_list)

In [ ]:

def LISA_markov_diagnostics(lm_list):
    
    
    for lm in lm_list:
#         lm_moves = pd.DataFrame({"Transitions":lm.transitions.flatten(), "Expected":lm.expected_t.flatten()}) 
#         lm_moves['Transitions'] = lm_moves['Transitions'].astype(int)
#         lm_moves.index = ["HH-HH","HH-LH","HH-LL","HH-HL", # Make index 16 possible move types
#                   "LH-HH","LH-LH","LH-LL","LH-HL",
#                   "LL-HH","LL-LH","LL-LL","LL-HL",
#                   "HL-HH","HL-LH","HL-LL","HL-HL"]
#         lm_moves['Residuals'] = lm_moves.Transitions - lm_moves.Expected # Calculate difference of observed - expected
#         print('Sum of LM transitions:')
#         print(sum([lm_moves['Residuals'][0], lm_moves['Residuals'][3],lm_moves['Residuals'][8],lm_moves['Residuals'][15]]))
        
        print("Chi2: %8.3f, p: %5.2f, dof: %d" % lm.chi_2)
#         ax = sns.heatmap(np.matrix([lm_moves.Transitions[:4].values,\
#         lm_moves.Transitions[4:8].values,\
#         lm_moves.Transitions[8:12].values,\
#         lm_moves.Transitions[12:].values]), cmap='Purples', 
#                     annot=True, annot_kws={'size':14}, cbar_kws={'label':"LISA Markov observed transitions"})
#         ax.figure.axes[-1].yaxis.label.set_size(18)
#         ax.figure.axes[-1].yaxis.label.set_size(18)
#         ax.set_yticklabels(labels=['HH', 'LH', 'LL', 'HL'], size=18);
#         ax.set_xticklabels(labels=['HH', 'LH', 'LL', 'HL'], size=18);
#         plt.ylabel("Transition from:", size=18)
#         plt.xlabel("Transition to:", size=18,);
#         plt.show();

#         ax = sns.heatmap(np.matrix([lm_moves.Residuals[:4].values,\
#         lm_moves.Residuals[4:8].values,\
#         lm_moves.Residuals[8:12].values,\
#         lm_moves.Residuals[12:].values]), cmap='coolwarm', 
#                     annot=True, annot_kws={'size':14}, cbar_kws={'label':"LISA Markov observed – expected transitions"})
#         ax.figure.axes[-1].yaxis.label.set_size(18)
#         ax.set_yticklabels(labels=['HH', 'LH', 'LL', 'HL'], size=18);
#         ax.set_xticklabels(labels=['HH', 'LH', 'LL', 'HL'], size=18);
#         plt.ylabel("Transition from:", size=18)
#         plt.xlabel("Transition to:", size=18);
#         plt.show();
        
        

In [ ]:
moving_clusters = GeoDataFrame(cluster_spots).rename(columns={0:'geometry'})
moving_clusters['hour'] = cluster_time
moving_clusters['geom_type'] = moving_clusters.geometry.apply(lambda x: type(x))
moving_clusters = moving_clusters.loc[moving_clusters['geom_type']==type(test)][['hour','geometry']]
for hour in moving_clusters.hour.unique():
    moving_clusters.loc[moving_clusters['hour']==hour].to_file('/Users/GeorgePyne/Desktop/shptest/{}hourclustertest.shp'.format(str(hour)), driver='ESRI Shapefile')

In [ ]:
xmin,ymin,xmax,ymax = gdf_points.total_bounds # Get point bounds
height = (xmax-xmin)/ 6 # Set grid resolution
grid = glt.make_grid(gdf_points,height, False) # Create unclipped grid
grid['counts'] = grid.geometry.apply(lambda x: pip_count(gdf_points,x)) # Aggregate points to grid